<a href="https://colab.research.google.com/github/diana-bsv/background-replacement/blob/main/background_replacement_with_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import os

from tqdm import tqdm

from PIL import Image
import cv2
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Resize, ToTensor

In [ ]:
# Загрузка модели и предобученных весов
!git clone https://github.com/NathanUA/U-2-Net.git

import sys
sys.path.append('./U-2-Net')

!wget 'https://huggingface.co/lilpotat/pytorch3d/resolve/346374a95673795896e94398d65700cb19199e31/u2net.pth' -O ./U-2-Net/u2net.pth

from model import U2NET

In [ ]:
# Загрузка данных для обработки
!wget "https://edu.tinkoff.ru/files/6fb0f21e-6f0f-4d7d-9a83-dab650e7ea10" -O data.zip
!unzip data.zip -d ./

os.mkdir(os.getcwd() + "/results")

In [4]:
# Модель https://github.com/NathanUA/U-2-Net.git
net = U2NET(3,1)

if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

# Загрузка предобученной модели
net.load_state_dict(torch.load("./U-2-Net/u2net.pth"));
net.to(device);
net.eval();

<ipython-input-4-32a2398e0f38>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("./U-2-Net/u2net.pth"));


In [5]:
class ImageDataset(Dataset):
  """ Датасет с изображениями товаров для обработки. Хранит путь к изображениям """
  def __init__(self, root_dir):
    self.image_paths = []

    for img_name in os.listdir(root_dir):
      if img_name[-4:] == ".jpg":
        self.image_paths.append('./sirius_data/' + img_name)

  def __len__(self):
    return len(self.image_paths)

  def __getitem__(self, idx):
    img_path = self.image_paths[idx]

    return img_path

In [6]:
# Работа с данными
transform = ToTensor()

dataset = ImageDataset(root_dir='./sirius_data')

dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=1)

In [7]:
def normPRED(d):
  """ Нормировка предсказаний модели """
  ma = torch.max(d)
  mi = torch.min(d)

  dn = (d-mi)/(ma-mi)

  return dn

def create_gradient(width, height):
  """ Создание вертикального затемнения изображения """
  gradient = np.linspace(255, 185, height, dtype=np.uint8)
  gradient = np.tile(gradient, (width, 1)).T

  return cv2.merge([gradient, gradient, gradient])

def create_background(width, height, color):
  """ Создание фонового изображения с шумом и градиентным затемнением """
  background = np.random.normal(loc=[max(0,color[0]-3),max(0,color[1]-3), max(0,color[2]-3)], scale=3, size=(height, width, 3)).astype(np.uint8)
  background = cv2.blur(background, (10, 10), 0)

  gradient = create_gradient(width, height)

  return cv2.addWeighted(background, 0.5, gradient, 0.5, 0)

def create_shadow(bg, mask):
  """ Создание изображения с тенью объекта (в форме маски)"""

  mask_blur = cv2.blur(mask, (45, 15))
  black = np.zeros(bg.shape, dtype=np.uint8)

  return (black + (1 - mask_blur) * bg).astype(np.uint8)



In [8]:
THRESHOLD = 0.15 # минимальное значение предсказания модели для попадания этого пикселя в маску
BACKGROUND_COLOR_RGB = [0, 150, 255] # желаемый цвет фона в формате RGB

for path in tqdm(dataloader):
  # Открытие изображения
  image = Image.open(path[0]).convert("RGB")
  image_t = transform(image).to(device)

  # Предсказания модели
  d1, d2, d3, d4, d5, d6, d7 = net(image_t.unsqueeze(0))
  predict = normPRED(d1[:,0,:,:].squeeze()) #нормирование предсказаний

  # Удаление пикселей с слишком низким значением вероятности принадлежности к объекту
  predict[predict <= THRESHOLD] = 0
  mask = predict.cpu().detach().numpy() #маска сегментированного изображения

  # Создание фона
  bg = create_background(image.width, image.height, BACKGROUND_COLOR_RGB)

  # Создание трехканальной маски сегментированного изображения
  mask_3d = cv2.merge([mask, mask, mask])

  # Наложение тени сегментированного изображения на его фон
  bg_shadow = create_shadow(bg, mask_3d)

  # Объединение сегментированного изображения и фона с его тенью
  final_image = (mask_3d * image + (1 - mask_3d) * bg_shadow).astype(np.uint8)

  # Сохранение изображения в файл results
  Image.fromarray(final_image).save('./results/' + path[0][14:])


  0%|          | 0/470 [00:00<?, ?it/s]/content/./U-2-Net/model/u2net.py:23: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  src = F.upsample(src,size=tar.shape[2:],mode='bilinear')
100%|██████████| 470/470 [00:32<00:00, 14.63it/s]
